In [3]:
! pip install -U scikit-learn

  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached scipy-1.9.3-cp39-cp39-win_amd64.whl (40.2 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


In [1]:
from sklearn.linear_model import ElasticNetCV #Linear regression with regularization, cross fold validation
from sklearn.linear_model import LinearRegression #Linear regression
import pandas as pd

In [17]:
# Convert country to country code
data = pd.read_csv('../normalized_data.csv')
# data['Country'] = data['Country'].astype('category')
# data['Country'] = data['Country'].cat.codes
# Drop NaN values
# print(data.iloc[0])
data = data[['Social support', 'Log GDP per capita', 'Healthy life expectancy at birth', 'Freedom to make life choices', 'Happiness_Score_Percentile']]

data = data.dropna()

# data.drop(['Country'], axis=1, inplace=True)
# data.drop(['Year'], axis=1, inplace=True)
# data.drop(['Happiness Score'], axis=1, inplace=True)
# data.drop(['Percentile Ranges'], axis=1, inplace=True)
print(data.iloc[250])

Social support                      0.748675
Log GDP per capita                  0.426987
Healthy life expectancy at birth    0.934696
Freedom to make life choices        1.096601
Happiness_Score_Percentile          1.000000
Name: 250, dtype: float64


In [18]:
# Split data into train, test
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)

#split datasets into x and y
y_train = train['Happiness_Score_Percentile']
x_train = train.drop(['Happiness_Score_Percentile'], axis=1)
y_test = test['Happiness_Score_Percentile']
x_test = test.drop(['Happiness_Score_Percentile'], axis=1)

assert len(x_train) == len(y_train)
assert len(x_test) == len(y_test)
assert len(x_train.iloc[0]) == len(x_test.iloc[0])
print(len(x_train), 'train examples')
print(len(x_test), 'test examples')
print(x_train.iloc[0])
print(y_train.iloc[0])

931 train examples
233 test examples
Social support                      0.208633
Log GDP per capita                 -1.055460
Healthy life expectancy at birth   -0.439945
Freedom to make life choices        0.581888
Name: 745, dtype: float64
5


In [19]:
#Find accuracy and error on test dataset
from sklearn.metrics import mean_squared_error

regr = ElasticNetCV(cv=5, random_state=0)
regr.fit(x_train, y_train)
print(regr.score(x_test, y_test))

y_pred = regr.predict(x_test)

# print(y_pred)
print("RMS: ", (mean_squared_error(y_test, y_pred))**2)
#Find accuracy
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_test, y_pred.round()))
# print("Accuracy: ", accuracy_score(y_test, y_pred))


0.7124847009020588
RMS:  2.118613781660754
Accuracy:  0.31759656652360513


In [20]:
# Standard Linear regression
reg = LinearRegression().fit(x_train, y_train)
y_pred = reg.predict(x_test)
print(reg.score(x_test, y_test))

#Find accuracy and error on test dataset
print("RMS: ", (mean_squared_error(y_test, y_pred))**2)
print("Accuracy: ", accuracy_score(y_test, y_pred.round()))

0.7125587095269714
RMS:  2.1175232274329168
Accuracy:  0.31759656652360513


In [23]:
#decision tree and visualization
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.tree import export_graphviz
import graphviz
from tqdm import tqdm

# clf = DecisionTreeClassifier(random_state=0, max_depth=10)
# clf = clf.fit(x_train, y_train)
# y_pred = clf.predict(x_test)
# print(clf.score(x_test, y_test))

# #visualize tree
# dot_data = tree.export_graphviz(clf, out_file=None)
# graph = graphviz.Source(dot_data)
# graph.render("happiness_depth10")

depths = [3,4,5,6,7,8,9,10]

for depth in tqdm(depths):
    clf = DecisionTreeClassifier(random_state=0, max_depth=depth)
    clf = clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print("Depth: ", depth, " Accuracy: ", clf.score(x_test, y_test))

    dot_data = tree.export_graphviz(clf, out_file=None)
    graph = graphviz.Source(dot_data)
    graph.render("tree_visualizations/happiness_depth" + str(depth))



 12%|█▎        | 1/8 [00:00<00:01,  4.50it/s]

Depth:  3  Accuracy:  0.3776824034334764
Depth:  4  Accuracy:  0.38626609442060084


 25%|██▌       | 2/8 [00:00<00:01,  4.50it/s]

Depth:  5  Accuracy:  0.34334763948497854


 38%|███▊      | 3/8 [00:00<00:01,  3.99it/s]

...

 75%|███████▌  | 6/8 [00:02<00:00,  2.47it/s]

Depth:  9  Accuracy:  0.4034334763948498


 88%|████████▊ | 7/8 [00:02<00:00,  2.14it/s]

Depth:  10  Accuracy:  0.4206008583690987


100%|██████████| 8/8 [00:03<00:00,  2.43it/s]
